### Converting the dataset.xlsx into datalog facts

In [4]:
import pyDatalog
import pandas as pd
import os

# Données Patients
patients_df = pd.read_excel("../data/dataset.xlsx", sheet_name="data")

# Création des fichiers Observed.facts et History.facts s'ils n'existent pas
observed_file_path = "../data/Observed.facts"
history_file_path = "../data/History.facts"

if not os.path.exists(observed_file_path) and not os.path.exists(history_file_path):
    with open(observed_file_path, "w") as observed_file, open(history_file_path, "w") as history_file:
        for _, row in patients_df.iterrows():
            patient_id = row["PatientID"]
            
            # Écrire dans le fichier Observed.facts si le symptôme observé est présent
            if pd.notna(row["Observed_Symptom"]):
                observed_file.write(f'"{patient_id}" "{row["Observed_Symptom"]}" {row["Observed_Week"]}\n')
            
            # Écrire dans le fichier History.facts si la condition historique est présente
            if pd.notna(row["History_Condition"]):
                history_file.write(f'"{patient_id}" "{row["History_Condition"]}" {row["History_Count"]}\n')
else:
    print("Les fichiers 'Observed.facts' et 'History.facts' existent déjà.")

Les fichiers 'Observed.facts' et 'History.facts' existent déjà.
